In [1]:
import json
import re
import time
from typing import List, Optional, Tuple

import evaluate
import kscope
from statistics import mean
from tqdm import tqdm
from utils import split_prompts_into_batches
import random
import numpy as np

/Users/yuchongzhang/Projects/PromptEngineeringLaboratory/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this notebook, we experiment with role-play prompting, as detailed in the paper "Better Zero-Shot Reasoning with Role-Play Prompting" (https://arxiv.org/pdf/2308.07702.pdf). The idea is fairly simple. Suppose we want the language model to perform a certain task T. If we "immerse" the model into the role of an expert on task T (or a role that is closely related to task T) through some conversational prompts, then the model might perform task T better. 

Following (roughly) the paper, we will compare role-play prompting with two other methods: Zero-shot prompting and Zero-shot CoT on the task of solving math word problems using the MultiArith dataset.

# Getting Started

There is a bit of documentation on how to interact with the large models [here](https://kaleidoscope-sdk.readthedocs.io/en/latest/). The relevant github links to the SDK are [here](https://github.com/VectorInstitute/kaleidoscope-sdk) and underlying code [here](https://github.com/VectorInstitute/kaleidoscope).

First we connect to the service through which we'll interact with the LLMs and see which models are avaiable to us

In [2]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=3001)

Show all supported models

In [60]:
client.models

['gpt2',
 'llama2-7b',
 'llama2-7b_chat',
 'llama2-13b',
 'llama2-13b_chat',
 'llama2-70b',
 'llama2-70b_chat',
 'falcon-7b',
 'falcon-40b',
 'sdxl-turbo']

Show all model instances that are currently active

In [75]:
client.model_instances

[{'id': 'e5ee7525-cc82-46f1-b7d0-aa25c3994571',
  'name': 'llama2-70b',
  'state': 'ACTIVE'}]

To start, we obtain a handle to a model. In this example, let's use the LLama 70B model.

In [76]:
model = client.load_model("llama2-70b")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

We need to configure the model to generate in the way we want it to. So we set a number of important parameters. For a discussion of the configuration parameters see: `src/reference_implementations/prompting_vector_llms/CONFIG_README.md`

In [61]:
long_generation_config = {"max_tokens": 60, "top_k": 4, "top_p": 1.0, "temperature": 0.5}

# MultiArith: Math Word Problems

MultiArith is a dataset consisting of math word problems. The following function is used to read the data from the raw json file. 

In [62]:
def data_reader(dataset_path: str) -> Tuple[List[str], List[str]]:
    questions = []
    answers = []
    with open(dataset_path) as f:
        json_data = json.load(f)
        for line in json_data:
            q = line["sQuestion"].strip()
            a = str(line["lSolutions"][0])
            if a[-2:] == ".0":
                a = a[:-2]
            questions.append(q)
            answers.append(a)
    return questions, answers

Here is an example question and the correct answer.

In [63]:
questions, answers = data_reader("resources/multi_arith_dataset/MultiArith.json")

print(f"Question: {questions[0]}")
print(f"Correct answer: {answers[0]}")

Question: For Halloween Debby and her sister combined the candy they received. Debby had 32 pieces of candy while her sister had 42. If they ate 35 pieces the first night, how many pieces do they have left?
Correct answer: 39


We randomly sample a subset and use it for evaluation since the whole dataset is quite large and prompting takes a while to run.

In [64]:
num_samples = 50
sample_questions, sample_answers = zip(*random.sample(list(zip(questions, answers)), num_samples))
sample_questions = list(sample_questions)
sample_answers = [int(answer) for answer in list(sample_answers)]

The targets for MultiArith are integers, and since the model would likely produce answers that are not integers, we need a way to extract a numerical answer from the model's response string. We perform this by another round of prompting. More precisely, for each question, after getting the answer generated by the LLM, we concatenate the question, answer, and an answer trigger together and feed them into the model again. 

For MultiArith specifically, we use the answer trigger "Therefore, the answer (arabic numerals) is" and we extract the first integer in the model's response to the second prompt as its prediction.

In [65]:
def extract_numerical_answer(response_str) -> int:
    response_str = response_str.replace(",", "")
    response_str = response_str.replace(".", "")
    numbers = [s for s in re.findall(r'-?\d+\.?\d*', response_str)]
    if len(numbers) > 0:
        return int(numbers[0])
    else:
        # if the model reponse does not contain any number, we just return a random integer.
        return random.randint(3000, 3100)


## Zero-Shot Prompting

First, let's try Zero-shot prompting with no CoT. We will begin by storing all the model's responses to the raw math questions.

In [66]:
zero_shot_responses = []
for sample_question in tqdm(sample_questions):
    generation = model.generate(sample_question, long_generation_config)
    zero_shot_responses.append(generation.generation)

100%|██████████| 50/50 [11:34<00:00, 13.88s/it]


In [67]:
i = 10
print("============ An example question and the model response ============")
print(f"Question: {sample_questions[i]}")
print(f"Correct answer: {sample_answers[i]}")
print(f"Model response:{zero_shot_responses[i]['sequences'][0]}")

============ An example question and the model response ============
Question: Katie's team won their dodgeball game and scored 12 points total. If Katie scored 4 of the points and everyone else scored 4 points each, how many players were on her team?
Correct answer: 2
Model response:
There were 4 players on Katie's team.
Katie's team won their dodgeball game and scored 12 points total. If Katie scored 4 of the points and everyone else scored 4 points each, how many players were on her team? There


We then concatenate each one of the model's responses with the answer trigger and feed the result into the model again in order to extract numerical answers.

In [68]:
multi_arith_answer_trigger = "Therefore, the answer (arabic numerals) is"
multi_arith_concatenated_prompts = []
second_generations = []
for sample_question, model_response in zip(sample_questions, zero_shot_responses):
    response_str = model_response['sequences'][0]
    concatenated_prompt = f"{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts.append(concatenated_prompt)
    second_generation = model.generate(concatenated_prompt, long_generation_config)
    second_generations.append(second_generation.generation)

In [69]:
print("============ An example question and the model response to the concatenated prompt ============")
print(f"Question: {sample_questions[i]}")
print(f"Correct answer: {sample_answers[i]}")
print(f"Second round prompt: {multi_arith_concatenated_prompts[i]}")
print(f"Model response:{second_generations[i]['sequences'][0]}")

============ An example question and the model response to the concatenated prompt ============
Question: Katie's team won their dodgeball game and scored 12 points total. If Katie scored 4 of the points and everyone else scored 4 points each, how many players were on her team?
Correct answer: 2
Second round prompt: Katie's team won their dodgeball game and scored 12 points total. If Katie scored 4 of the points and everyone else scored 4 points each, how many players were on her team?'
'
There were 4 players on Katie's team.
Katie's team won their dodgeball game and scored 12 points total. If Katie scored 4 of the points and everyone else scored 4 points each, how many players were on her team? There'
'Therefore, the answer (arabic numerals) is
Model response:: 4
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'



Finally, we compare the extracted numerical answers with the correct answers to compute the accuracy.

In [70]:

final_answers = [extract_numerical_answer(second_generation['sequences'][0]) for second_generation in second_generations]
accuracy = np.sum(np.array(final_answers) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.16


The performance is not very good. Next, we will follow the same evaluation procedure to test the other methods.

## Zero-shot CoT

Now we try Zero-shot CoT as another baseline for comparison. This is pretty similar to the vanilla zero-shot approach above, except we append the sentence "Let's think step by step" to the end of each prompt to encourage the model to perform CoT.

In [72]:
cot_post_prompt = "Let's think step by step."

In [73]:
zero_shot_cot_responses = []
for sample_question in tqdm(sample_questions):
    prompt = f"{sample_question}\n{cot_post_prompt}"
    generation = model.generate(prompt, long_generation_config)
    zero_shot_cot_responses.append(generation.generation)

100%|██████████| 50/50 [11:07<00:00, 13.35s/it]


In [77]:
multi_arith_concatenated_prompts_cot = []
second_generations_cot = []
for sample_question, model_response in zip(sample_questions, zero_shot_cot_responses):
    response_str = model_response['sequences'][0]
    concatenated_prompt = f"{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts_cot.append(concatenated_prompt)
    second_generation = model.generate(concatenated_prompt, long_generation_config)
    second_generations_cot.append(second_generation.generation)

In [78]:

final_answers_cot = [extract_numerical_answer(second_generation['sequences'][0]) for second_generation in second_generations_cot]
accuracy = np.sum(np.array(final_answers_cot) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.52


In [79]:
print(final_answers_cot)
print(sample_answers)

[4, 5, 3046, 4, 15, 3054, 11, 20, 12, 3055, 4, 3002, 55, 32, 12, 45, 43, 3032, 10, 35, 27, 1, 6, 16, 60, 48, 19, 36, 64, 8, 34, 41, 9, 60, 8, 27, 29, 6, 63, 6, 36, 11, 33, 3051, 49, 86, 6, 5, 3035, 73]
[4, 45, 3, 4, 15, 4, 11, 4, 36, 9, 2, 64, 51, 32, 9, 45, 43, 18, 10, 35, 27, 5, 6, 16, 58, 48, 49, 36, 80, 8, 34, 41, 9, 60, 8, 21, 29, 6, 63, 12, 21, 6, 27, 4, 63, 86, 6, 2, 3, 73]


We can see that by adding the simple sentence "Let's think step by step.", CoT induces a significant improvement in performance. Now let's see whether role-play prompting works better.

## Role-play Prompting

We consider three different role-play prompts, representing three different levels of immersion. 

In the first level, we simply inform the model of the role it is expected to play before asking the question.

In the second level, immersion is enhanced by adding complementary descriptions of the role in the prompt.

In the third level, we append to the end of the level-2 prompt a "response" in which the model acknowledges the role it plays. Because we are using Llama-70b rather than Llamo-70b-chat, it is tricky to induce this kind of response from the model. So instead, we just artificially create the response and concatenate it with the level-2 prompt.


In [80]:
role_prompt_level1 = "From now on, you are a math teacher. Please answer the following question"
role_prompt_level2 = "From now on, you are an excellent math teacher and always teach your students math problems correctly. I am one of your students and ask you the following question."
role_prompt_level3 = "From now on, you are an excellent math teacher and always teach your students math problems correctly. And I am one of your students. "

### Level 1:

In [81]:
level1_responses = []
for sample_question in tqdm(sample_questions):
    prompt = f"{role_prompt_level1}\n{sample_question}"
    generation = model.generate(prompt, long_generation_config)
    level1_responses.append(generation.generation)

100%|██████████| 50/50 [12:17<00:00, 14.75s/it]


In [82]:
multi_arith_concatenated_prompts_level1 = []
second_generations_level1 = []
for sample_question, model_response in zip(sample_questions, level1_responses):
    response_str = model_response['sequences'][0]
    concatenated_prompt = f"{role_prompt_level1}\n{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts_level1.append(concatenated_prompt)
    second_generation = model.generate(concatenated_prompt, long_generation_config)
    second_generations_level1.append(second_generation.generation)

In [83]:

final_answers_level1 = [extract_numerical_answer(second_generation['sequences'][0]) for second_generation in second_generations_level1]
accuracy = np.sum(np.array(final_answers_level1) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.14


Simply telling the model the role we wish it to play does not seem to help. We can try printing out some of the model's responses to our role-play prompt to see what is happening.

In [98]:
print(level1_responses[6]['sequences'][0])
print(final_answers_level1)
print(sample_answers)


A. 10 books
B. 13 books
C. 14 books
D. 17 books
E. 21 books
I would say C, but I'm not a math teacher.
C, but I'm not a math
[6, 45, 4, 11, 13, 3004, 14, 6, 100, 7, 3, 16, 5, 32, 16, 85, 46, 18, 11, 3072, 27, 6, 30, 11, 3042, 48, 3079, 18, 80, 13, 20, 14, 27, 23, 18, 12, 29, 5, 9, 15, 36, 3, 33, 3058, 3087, 90, 10, 3, 4, 43]
[4, 45, 3, 4, 15, 4, 11, 4, 36, 9, 2, 64, 51, 32, 9, 45, 43, 18, 10, 35, 27, 5, 6, 16, 58, 48, 49, 36, 80, 8, 34, 41, 9, 60, 8, 21, 29, 6, 63, 12, 21, 6, 27, 4, 63, 86, 6, 2, 3, 73]


### Level 2

In [85]:
level2_responses = []
for sample_question in tqdm(sample_questions):
    prompt = f"{role_prompt_level2}\n{sample_question}"
    generation = model.generate(prompt, long_generation_config)
    level2_responses.append(generation.generation)

100%|██████████| 50/50 [12:25<00:00, 14.92s/it]


In [86]:
multi_arith_concatenated_prompts_level2 = []
second_generations_level2 = []
for sample_question, model_response in zip(sample_questions, level2_responses):
    response_str = model_response['sequences'][0]
    concatenated_prompt = f"{role_prompt_level2}\n{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts_level2.append(concatenated_prompt)
    second_generation = model.generate(concatenated_prompt, long_generation_config)
    second_generations_level2.append(second_generation.generation)

In [87]:
final_answers_level2 = [extract_numerical_answer(second_generation['sequences'][0]) for second_generation in second_generations_level2]
accuracy = np.sum(np.array(final_answers_level2) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.16


It appears that adding more detailed descriptions to the role slightly increases the performance. But the improvement is very negligible: from level 1 to level 2, the accuracy increased from 0.14 to 0.16.

In [97]:
print(level2_responses[6]['sequences'][0])
print(final_answers_level2)
print(sample_answers)


I am sure you would tell me that there are 11 books in the bin.
But what if I ask you the following question.
A book store had 20 books in the bargin bin. If they sold 10 books, but then put 10 more
[14, 54, 3036, 3073, 5, 4, 10, 6, 6, 3029, 3099, 7, 48, 3052, 11, 17, 46, 3037, 40, 3037, 34, 9, 3, 16, 37, 6, 65, 36, 80, 16, 30, 1, 13, 3005, 2, 18, 29, 6, 36, 3, 3031, 3025, 28, 3094, 98, 86, 6, 3083, 3125, 75]
[4, 45, 3, 4, 15, 4, 11, 4, 36, 9, 2, 64, 51, 32, 9, 45, 43, 18, 10, 35, 27, 5, 6, 16, 58, 48, 49, 36, 80, 8, 34, 41, 9, 60, 8, 21, 29, 6, 63, 12, 21, 6, 27, 4, 63, 86, 6, 2, 3, 73]


### Level 3

Finally, let's see how well level-3 role-paly prompting work.

In [89]:
def discard_after_last_period(input_string: str) -> str:
    # Find the last period in the string
    last_period_index = input_string.rfind('.')

    # If a period is found, discard everything after it
    if last_period_index != -1:
        result_string = input_string[:last_period_index + 1]
    else:
        # If no period is found, return the original string
        result_string = input_string

    return result_string

Below is the artificial response we will append to the role prompt. As mentioned before, it is tricky to induce this kind of response from the model. We print out an example of the mode's actual response.

In [90]:
role_prompt_level3_follow_up = "That’s great to hear! As your math teacher, I’ll do my best to explain mathematical concepts correctly so that you can understand them easily. Feel free to ask any math problems or questions you have, and I’ll be glad to assist you. Let’s dive into the world of mathematics and explore its wonders together!"

In [91]:
generation_level3_role_prompt = model.generate(role_prompt_level3, long_generation_config)
print(generation_level3_role_prompt.generation['sequences'][0])
role_prompt_response_level3 = discard_after_last_period(generation_level3_role_prompt.generation['sequences'][0])

😀
It’s so nice to be your student.
I’m so happy that you are my student.
I’m so happy that you are my student. 😀
I’m so happy that you are my student. 😀 😀
I’m so happy that you are my student. 😀 😀 😀
I’m so happy that


In [92]:
level3_responses = []
for sample_question in tqdm(sample_questions):
    prompt = f"{role_prompt_level3}\n{role_prompt_level3_follow_up}\n{sample_question}"
    generation = model.generate(prompt, long_generation_config)
    level3_responses.append(generation.generation)

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [19:57<00:00, 23.96s/it]


In [93]:
multi_arith_concatenated_prompts_level3 = []
second_generations_level3 = []
for sample_question, model_response in zip(sample_questions, level3_responses):
    response_str = model_response['sequences'][0]
    concatenated_prompt = f"{role_prompt_level3}\n{role_prompt_level3_follow_up}\n{sample_question}'\n'{response_str}'\n'{multi_arith_answer_trigger}"
    multi_arith_concatenated_prompts_level3.append(concatenated_prompt)
    second_generation = model.generate(concatenated_prompt, long_generation_config)
    second_generations_level3.append(second_generation.generation)

In [94]:
final_answers_level3 = [extract_numerical_answer(second_generation['sequences'][0]) for second_generation in second_generations_level3]
accuracy = np.sum(np.array(final_answers_level3) == np.array(sample_answers)) / len(sample_answers)
print(f"Accuracy: {accuracy}")

Accuracy: 0.42


Adding the fake response in which the model acknowledges its role improves the performance quite markedly. We got better accuracy than Zero-shot or the previous two immersion levels. But the accuracy is still worse than Zero-shot CoT. 

In [99]:
print(second_generations_level3[6]['sequences'][0])
print(final_answers_level3)
print(sample_answers)

11'
'Therefore, the answer (roman numerals) is XI'
'Therefore, the answer (greek numerals) is ΙΙ'
'Therefore, the answer (hebrew numerals) is א'
'Therefore, the answer (binary) is 1011'
'Therefore, the answer (octal) is 13'
'Therefore, the answer (hexadec
[5, 9, 3, 4, 15, 4, 11, 4, 48, 15, 7, 7, 53, 32, 9, 65, 43, -8, 1, 35, 13, 3, 1, 16, 58, 16, 49, 3031, 80, 3079, 11, 31, 11, 60, 24, 27, 34, 6, 9, 12, 21, 5, 90, 4, 9, 86, 5, 2, 4, 43]
[4, 45, 3, 4, 15, 4, 11, 4, 36, 9, 2, 64, 51, 32, 9, 45, 43, 18, 10, 35, 27, 5, 6, 16, 58, 48, 49, 36, 80, 8, 34, 41, 9, 60, 8, 21, 29, 6, 63, 12, 21, 6, 27, 4, 63, 86, 6, 2, 3, 73]


# Some Concluding Thoughts

The original paper provided some evidence that role-play prompting can work better than CoT in many tasks, but in their case the model was able to acknowledge its role on its own, so there was no need to artifically create a fake response. This could be the reason why role-play prompting does not work as well as CoT in our case. Intuitively, one might say that if the model acknowledges its role on its own, then the responses which come after that naturally extend this context, so in some sense it is similar to CoT. 

We can also observe that in level 1 and 2, the model gave some incoherent responses which suggest it did not immerse itself in the role of a math teacher. This could be the reason why these two levels failed to improve upon Zero-shot. The model we are using here (Llama-70b) has not been fine-tuned for chat purposes, so it is harder to induce conversation-like behaviour from the model, which can be important for role-play prompting to work. A natural experiment one might want to run to verify this idea is to use a model that has been fine-tuned for chat purposes.
